<a href="https://colab.research.google.com/github/axiomhaidarcreate/project-AI-/blob/main/Copy_of_lstm_ddos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dhoogla_cicddos2019_path = kagglehub.dataset_download('dhoogla/cicddos2019')
azharzen_lstm_ddos_model_path = kagglehub.dataset_download('azharzen/lstm-ddos-model')

print('Data source import complete.')

Data source import complete.


# Load Data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import resample
from sklearn import preprocessing

import os
dataset_path = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.csv'):
            dfp = os.path.join(dirname, filename)
            dataset_path.append(dfp)

In [ ]:
print(dataset_path)

[]


In [ ]:
if len(dataset_path) > 1:
    cols = list(pd.read_csv(dataset_path[1], nrows=1))
else:
    print("Error: dataset_path does not contain enough elements.")

Error: dataset_path does not contain enough elements.


In [ ]:
if not dataset_path:
    print("Error: dataset_path is empty.")
else:
    print("Dataset paths:", dataset_path)

Error: dataset_path is empty.


In [ ]:
if not dataset_path:
    print("Error: dataset_path is empty.")
else:
    print("Dataset paths:", dataset_path)

Error: dataset_path is empty.


In [ ]:
import os

for path in dataset_path:
    if os.path.exists(path):
        print(f"File exists: {path}")
    else:
        print(f"File not found: {path}")

In [ ]:
print("Dataset paths:", dataset_path)

Dataset paths: []


In [ ]:
import pandas as pd

# إنشاء ملفات CSV وهمية
df1 = pd.DataFrame({"A": [1, 2], "B": [3, 4]})
df1.to_csv("path_to_first_dataset.csv", index=False)

df2 = pd.DataFrame({"C": [5, 6], "D": [7, 8]})
df2.to_csv("path_to_second_dataset.csv", index=False)

dataset_path = ["path_to_first_dataset.csv", "path_to_second_dataset.csv"]
print("Test files created.")

Test files created.


In [ ]:
print("Dataset paths:", dataset_path)

Dataset paths: ['path_to_first_dataset.csv', 'path_to_second_dataset.csv']


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import resample
from sklearn import preprocessing

import os
dataset_path = []

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.csv'):
            dfp = os.path.join(dirname, filename)
            dataset_path.append(dfp)

In [ ]:
# mult = 5
cols = list(pd.read_csv(dataset_path[1], nrows=1))

def load_file(path):
    # data = pd.read_csv(path, sep=',')
    data = pd.read_csv(path,
                   usecols =[i for i in cols if i != " Source IP"
                             and i != ' Destination IP' and i != 'Flow ID'
                             and i != 'SimillarHTTP' and i != 'Unnamed: 0'])

    return data

IndexError: list index out of range

In [ ]:
samples = pd.concat([load_file(dfp) for dfp in dataset_path], ignore_index=True)

ValueError: Usecols do not match columns, columns expected but not found: ['C', 'D']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Count the occurrences of each label
label_counts = samples[' Label'].value_counts()

# Create a bar plot to visualize the label counts
plt.figure(figsize=(10, 6))
label_counts.plot(kind='bar')
plt.title('Comparison of Label Column')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
samples

# preprocess

In [ ]:
# samples = samples_raw
def string2numeric_hash(text):
    import hashlib
    return int(hashlib.md5(text).hexdigest()[:8], 16)

# Flows Packet/s e Bytes/s - Replace infinity by 0
samples = samples.replace('Infinity','0')
samples = samples.replace(np.inf,0)
#samples = samples.replace('nan','0')
samples[' Flow Packets/s'] = pd.to_numeric(samples[' Flow Packets/s'])

samples['Flow Bytes/s'] = samples['Flow Bytes/s'].fillna(0)
samples['Flow Bytes/s'] = pd.to_numeric(samples['Flow Bytes/s'])


#Label
from sklearn.preprocessing import LabelEncoder
# Create a label encoder object
label_encoder = LabelEncoder()

# Fit the label encoder to the label column and transform the labels
samples[' Label'] = label_encoder.fit_transform(samples[' Label'])

# Get the mapping between original labels and encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# Print the mapping
for label, encoded_value in label_mapping.items():
    print(f"Label: {label} - Encoded Value: {encoded_value}")

#Timestamp - Drop day, then convert hour, minute and seconds to hashing
colunaTime = pd.DataFrame(samples[' Timestamp'].str.split(' ',1).tolist(), columns = ['dia','horas'])
colunaTime = pd.DataFrame(colunaTime['horas'].str.split('.',1).tolist(),columns = ['horas','milisec'])
stringHoras = pd.DataFrame(colunaTime['horas'].str.encode('utf-8'))
samples[' Timestamp'] = pd.DataFrame(stringHoras['horas'].apply(string2numeric_hash))#colunaTime['horas']
del colunaTime,stringHoras


print('Training data processed')

In [ ]:
samples

In [ ]:
# Import required libraries
from keras.models import Sequential

from keras.layers import Dense,Embedding,Dropout,Flatten,MaxPooling1D,LSTM


In [ ]:
def LSTM_model(input_size):

    # Initialize the constructor
    model = Sequential()

    model.add(LSTM(32,input_shape=(input_size,1), return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(18, activation='softmax'))  # Update units and activation function

    print(model.summary())

    return model

In [ ]:
def train_test(samples):
    from sklearn.model_selection import train_test_split
    import numpy as np

    # Specify the data
    X=samples.iloc[:,0:(samples.shape[1]-1)]

    # Specify the target labels and flatten the array
    #y= np.ravel(amostras.type)
    y= samples.iloc[:,-1]

    # Split the data up in train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    return X_train, X_test, y_train, y_test

# X_train, X_test, y_train, y_test = train_test(samples)

In [ ]:
def format_3d(df):

    X = np.array(df)
    return np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
# compile and train learning model

def compile_train(model,X_train,y_train,deep=True, epochs=10):

    if(deep==True):
        import matplotlib.pyplot as plt
#         optimizer = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=0.001)
#         model_lstm2.compile(optimizer=optimizer)

#         model.compile(loss='binary_crossentropy',
#                       optimizer='adam',
#                       metrics=['accuracy'])
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        history = model.fit(X_train, y_train,epochs=epochs, batch_size=512, verbose=1)
        #model.fit(X_train, y_train,epochs=3)

        # summarize history for accuracy
        plt.plot(history.history['accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train'], loc='upper left')
        plt.show()
        # summarize history for loss
        plt.plot(history.history['loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train'], loc='upper left')
        plt.show()

        print(model.metrics_names)

    else:
        model.fit(X_train, y_train) #SVM, LR, GD

    print('Model Compiled and Trained')
    return model

In [ ]:
# Save model and weights

def save_model(model,name):
    from keras.models import model_from_json

    arq_json = 'Models/' + name + '.json'
    model_json = model.to_json()
    with open(arq_json,"w") as json_file:
        json_file.write(model_json)

    arq_h5 = 'Models/' + name + '.h5'
    model.save_weights(arq_h5)
    print('Model Saved')

def load_model(name):
    from keras.models import model_from_json

    arq_json = 'Models/' + name + '.json'
    json_file = open(arq_json,'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    arq_h5 = 'Models/' + name + '.h5'
    loaded_model.load_weights(arq_h5)

    print('Model loaded')

    return loaded_model


# Resample

In [ ]:
# UPSAMPLE

X_train, X_test, y_train, y_test = train_test(samples)


#junta novamente pra aumentar o numero de normais
X = pd.concat([X_train, y_train], axis=1)

from imblearn.over_sampling import SMOTE

X_train, y_train = SMOTE(random_state=42).fit_resample(X, X[' Label'])
X_train = pd.DataFrame(X_train, columns=X_train.columns)

X_train=X_train.drop(' Label', axis=1)
input_size = (X_train.shape[1], 1)

del X

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Count the occurrences of each label
label_counts = y_train.value_counts()

# Create a bar plot to visualize the label counts
plt.figure(figsize=(10, 6))
label_counts.plot(kind='bar')
plt.title('Comparison of Label Column')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
from keras.utils import to_categorical

# labels = to_categorical(samples[' Label'])
y_train= to_categorical(y_train)
y_test= to_categorical(y_test)

# Training

In [ ]:
model_lstm2 = LSTM_model(82)
model_lstm2 = compile_train(model_lstm2,format_3d(X_train.astype(np.float32)),y_train.astype(np.float32), epochs=50)


## save model

In [ ]:
!mkdir Models

In [ ]:
!touch 'Models/lstm_50e.json'
save_model(model_lstm2,"lstm_50e")

In [ ]:
del model_lstm2

## Load model

In [ ]:
!cp -r /kaggle/input/lstm-ddos-model/* /kaggle/working/Models/

In [ ]:
!ls -lah Models

In [ ]:
model_lstm = load_model('lstm_50e')
# model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Prediction

In [ ]:
y_pred = model_lstm.predict(format_3d(X_test))

y_pred = y_pred.round()


In [ ]:
y_pred_label = np.argmax(y_pred, axis=1)
y_test_label = np.argmax(y_test, axis=1)

## evaluation

In [ ]:
# score = model_lstm2.evaluate(X_test, y_test,verbose=1)

# print(score)

from sklearn.metrics import accuracy_score
accuracy_score(y_test_label, y_pred_label)

In [ ]:
from sklearn.metrics import classification_report
classification_report(y_test_label, y_pred_label)

In [ ]:
# '              precision    recall  f1-score   support

#            0       0.47      0.99      0.64     37272
#            1       0.70      0.34      0.45     10007
#            2       0.00      0.00      0.00      4819
#            3       0.43      0.45      0.44      5937
#            4       0.99      0.97      0.98     42689
#            5       0.59      0.31      0.40      5087
#            6       0.41      0.76      0.53      4498
#            7       0.53      0.27      0.36      6316
#            8       0.00      0.00      0.00     13571
#            9       0.60      0.14      0.22      8315
#           10       0.64      0.11      0.19      5311
#           11       0.74      0.53      0.62     14089
#           12       1.00      0.99      0.99    118000
#           13       0.99      0.99      0.99     75196
#           14       0.67      0.66      0.66     11050
#           15       0.84      0.87      0.86     10910
#           16       0.07      0.84      0.12        25
#           17       0.11      0.96      0.20        23

#     accuracy                           0.83    373115
#    macro avg       0.54      0.57      0.48    373115
# weighted avg       0.82      0.83      0.80    373115
# '


In [ ]:
# !apt -qqqq install aria2

In [ ]:
# !mkdir dataset
# !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d /kaggle/working/dataset -o PCAP-01-12_0750-0818.zip  http://205.174.165.80/CICDataset/CICDDoS2019/Dataset/PCAPs/01-12/PCAP-01-12_0750-0818.zip

In [ ]:
# !unzip -d /kaggle ./dataset/PCAP-01-12_0750-0818.zip

In [ ]:
# !ls -lah /kaggle

In [ ]:
# def parse_pcap(path):
#     #TODO dunno how to do

# Contoh

In [ ]:
# anngap file pcap sudah diextract menjadi csv
def detect_anomaly(csv_path, model):
    # read data
    data = pd.read_csv(csv_path,
               usecols =[i for i in cols if i != ' Destination IP' and i != 'Flow ID'
                         and i != 'SimillarHTTP' and i != 'Unnamed: 0'])
    source_ip = data[' Source IP']
    data = data.drop(" Source IP", axis=1)

    # only for data from dataset
    data = data.drop(' Label', axis=1)

    # preprocess
    def string2numeric_hash(text):
        import hashlib
        return int(hashlib.md5(text).hexdigest()[:8], 16)

    # Flows Packet/s e Bytes/s - Replace infinity by 0
    data = data.replace('Infinity','0')
    data = data.replace(np.inf,0)
    #data = data.replace('nan','0')
    data[' Flow Packets/s'] = pd.to_numeric(data[' Flow Packets/s'])

    data['Flow Bytes/s'] = data['Flow Bytes/s'].fillna(0)
    data['Flow Bytes/s'] = pd.to_numeric(data['Flow Bytes/s'])

    #Timestamp - Drop day, then convert hour, minute and seconds to hashing
    colunaTime = pd.DataFrame(data[' Timestamp'].str.split(' ',1).tolist(), columns = ['dia','horas'])
    colunaTime = pd.DataFrame(colunaTime['horas'].str.split('.',1).tolist(),columns = ['horas','milisec'])
    stringHoras = pd.DataFrame(colunaTime['horas'].str.encode('utf-8'))
    data[' Timestamp'] = pd.DataFrame(stringHoras['horas'].apply(string2numeric_hash))#colunaTime['horas']
    del colunaTime,stringHoras

    predictions = model.predict(data)
    predictions = np.argmax(predictions, axis=1)

    # Create a DataFrame of anomalies
#     anomaly_df = pd.DataFrame()
#     for i in range(len(predictions)):
#         if predictions[i] != "0":
#             anomaly_df = anomaly_df.append({' Source IP': source_ip[i], "Label": predictions[i]}, ignore_index=True)

    anomaly_df = np.array([])
    for i in range(len(predictions)):
        if predictions[i] != 0:
            anomaly_df = np.append(anomaly_df, [source_ip[i], predictions[i]])

    # Print the IP address of the unique anomalies
#     print(anomaly_df["IP"].unique())
    anomaly_df = anomaly_df.reshape(-1, 2)
    label_map = {
        '0': "BENIGN",
        '1': "DrDoS_DNS",
        '2': "DrDoS_LDAP",
        '3': "DrDoS_MSSQL",
        '4': "DrDoS_NTP",
        '5': "DrDoS_NetBIOS",
        '6': "DrDoS_SNMP",
        '7': "DrDoS_UDP",
        '8': "LDAP",
        '9': "MSSQL",
        '10': "NetBIOS",
        '11': "Portmap",
        '12': "Syn",
        '13': "TFTP",
        '14': "UDP",
        '15': "UDP-lag",
        '16': "UDPLag",
        '17': "WebDDoS",
    }
    unique_values = np.unique(anomaly_df, axis=0)

    # Print the results in a fancy format.
    print('Source IP | anomaly')
    print('===================')
    for row in unique_values:
        print(f"{row[0]} | {label_map[row[1]]}")

In [ ]:
detect_anomaly('/kaggle/input/cicddos2019/DNS-testing.csv', model_lstm)